In [1]:
import numpy as np
from src.scicode.parse.parse import process_hdf5_to_tuple

# Function to calculate the optical binding force
def binding_force(P, phi, R, l, w, a, n):
    '''
    Function to calculate the optical binding force between two trapped nanospheres.

    Input
    -----
    P : list of length 2
        Power of the two optical traps.
    phi : float
        Polarization direction of the optical traps.
    R : float
        Distance between the trapped nanospheres.
    l : float
        Wavelength of the optical traps.
    w : float
        Beam waist of the optical traps.
    a : float
        Radius of the trapped microspheres.
    n : float
        Refractive index of the trapped microspheres.

    Output
    ------
    F : float
        The optical binding force between two trapped nanospheres.
    '''

    # Constants
    c = 3e8  # Speed of light (m/s)
    eps_0 = 8.854e-12  # Vacuum permittivity (F/m)
    k = 2 * np.pi / l  # Wave number

    # Scalar polarizability
    alpha = (4 * np.pi * eps_0 * a**3 * (n**2 - 1)) / (n**2 + 2)

    # Electric fields of the two optical traps
    E1 = np.sqrt(4 * P[0] / (np.pi * w**2 * eps_0 * c))
    E2 = np.sqrt(4 * P[1] / (np.pi * w**2 * eps_0 * c))

    # Fx calculation components
    cos_phi2 = np.cos(phi)**2
    sin_phi2 = np.sin(phi)**2

    Fxx = (
        2 * alpha**2 * E1 * E2 * cos_phi2 / (8 * np.pi * eps_0 * R**4)
        * (-3 * np.cos(k * R) - 3 * k * R * np.sin(k * R) + (k * R)**2 * np.cos(k * R))
    )
    Fxy = (
        alpha**2 * E1 * E2 * sin_phi2 / (8 * np.pi * eps_0 * R**4)
        * (3 * np.cos(k * R) + 3 * k * R * np.sin(k * R) - 2 * (k * R)**2 * np.cos(k * R) - (k * R)**3 * np.sin(k * R))
    )

    # Total force
    F = Fxx + Fxy
    return F

# Function to load test outputs
def load_test_outputs():
    # Define the HDF5 file path relative to the repository root
    h5_file_path = "eval/data/test_data.h5"

    # Set the global variable required by process_hdf5_to_tuple
    global H5PY_FILE
    H5PY_FILE = h5_file_path

    # Load test outputs for the specified step and number of tests
    step_id = "32.1"  # Replace with the correct step identifier
    test_num = 3       # Number of test cases
    return process_hdf5_to_tuple(step_id, test_num)

# Main script to test the function
if __name__ == "__main__":
    # Load the target outputs from the HDF5 file
    test_outputs = load_test_outputs()
    print("Loaded Test Outputs:", test_outputs)

    # Define the test cases
    test_cases = [
        {"P": [10000000, 100000000], "phi": 0, "R": 1550e-9, "l": 1550e-9, "w": 600e-9, "a": 100e-9, "n": 1.444},
        {"P": [10000000, 100000000], "phi": np.pi / 2, "R": 1550e-9, "l": 1550e-9, "w": 600e-9, "a": 100e-9, "n": 1.444},
        {"P": [1000000000, 1000000000], "phi": np.pi / 4, "R": 1550e-9, "l": 1550e-9, "w": 600e-9, "a": 100e-9, "n": 1.444},
    ]

    # Numerical tolerance settings
    rtol = 1e-4  # Relative tolerance
    atol = 1e-4  # Absolute tolerance

    # Run the test cases
    for i, test_case in enumerate(test_cases):
        # Calculate the optical binding force
        calculated_force = binding_force(**test_case)

        # Get the target force from the loaded test outputs
        target_force = test_outputs[i]

        # Check if the calculated force matches the target within the specified tolerance
        assert np.allclose(calculated_force, target_force, rtol=rtol, atol=atol), (
            f"Test case {i+1} failed! "
            f"Calculated: {calculated_force}, Target: {target_force}"
        )

        # Print the results
        print(f"Test case {i+1} passed: Calculated Force = {calculated_force}, Target = {target_force}")


Loaded Test Outputs: [2.090553478330249e-06, -2.17651746535146e-06, -1.3592099786808985e-06]
Test case 1 passed: Calculated Force = 2.0891072194969183e-06, Target = 2.090553478330249e-06
Test case 2 passed: Calculated Force = -2.1750117360588147e-06, Target = -2.17651746535146e-06
Test case 3 passed: Calculated Force = -1.3582696681562503e-06, Target = -1.3592099786808985e-06
